In [4]:
run functions_matching.ipynb


/eos/user/i/icortino/lhcbStyle.C:7:9: error: redefinition of 'black'
  Int_t black  = 1;
        ^
/eos/user/i/icortino/lhcbStyle.C:7:9: note: previous definition is here
  Int_t black  = 1;
        ^
fatal error: cannot open file 'input_line_76': No such file or directory


In [2]:
run configurations_matching.ipynb

In [5]:
smearing = True
isUTpresent = True
print 'smearing: ', smearing 
print 'is UT present: ', isUTpresent

percentage_list = range(99,59,-2)
#fill the list of indexes of the percentage (will be needed for the chi2 thresholds)
index_perc_list = []
for percentage in percentage_list:
    index_perc_list.append(perc_list.index(percentage))

##### INITIALISE VARIABLES #####

timestart = time.time()

nLongTracks = 0
nUTUTTracks = 0;nVeloUTTracks=0; nVeloTracks=0
nUPTracks = 0
real=[0.]*len(percentage_list);fake=[0.]*len(percentage_list) #real/fake minimums below threshold
realbelow=[0.]*len(percentage_list);fakebelow=[0.]*len(percentage_list) #how many times a real/fake is below threshold (hope for not many fakes) 
chi_below_thr =[0.]*len(percentage_list)  #tot number of chi2 below threshold
missed_real=[0.]*len(percentage_list);missed_fake=[0.]*len(percentage_list) #real/fake minimums above threshold (->missed)
#decrease threshold to increase missed_fake (but then also missed_real increases -> compromise)

nentries = ch1.GetEntries()


#plots for strategy min chi2
chi2R_true_list=[]
chi2R_list=[]
chi2F_list=[]

pR_true_list=[]
pR_list=[]
pF_list=[]

xzt0R_true_list=[]
xzt0R_list=[]
xzt0F_list=[]
yzt0R_true_list=[]
yzt0R_list=[]
yzt0F_list=[]

dxR_true_list=[]
dxR_list=[]
dxF_list=[]

dyR_true_list=[]
dyR_list=[]
dyF_list=[]

dtxR_true_list=[]
dtxR_list=[]
dtxF_list=[]

dtyR_true_list=[]
dtyR_list=[]
dtyF_list=[]

#plots for strategy cut below threshold
percbelow_th_list=[]
percF_th_list=[]

##### LOOP #####    
for event in ch1:


    ChisquareTry=[]
    Chi2Try_entryNumbers=[]
    pTry=[]
    xzt0Try=[]
    yzt0Try=[]
    dxTry=[]
    dyTry=[]
    dtxTry=[]
    dtyTry=[]

    if event.p-event.pz != 0:
        eta = 0.5*np.log((event.p+event.pz)/(event.p-event.pz))
    else:
        eta = -1

    if event.p > 5000 and 2<eta<5:          
        velolistsxyz = removezeros(event.HitVeloXpos,event.HitVeloYpos,event.HitVeloZpos)
        velolistsx = velolistsxyz[0];velolistsy = velolistsxyz[1];velolistsz = velolistsxyz[2]

        utlistsxyz = removezeros(event.HitUTXpos,event.HitUTYpos,event.HitUTZpos)
        utlistsx = utlistsxyz[0];utlistsy = utlistsxyz[1];utlistsz = utlistsxyz[2]

        eventconfig = whichconfig(utlistsz,velolistsz)
        if eventconfig != -1:
            nUPTracks +=1

            if eventconfig == 'VeloUT':
                nVeloUTTracks+=1
            elif eventconfig == 'UTUT':
                nUTUTTracks+=1
            elif eventconfig == 'VeloVelo':
                nVeloTracks+=1
        
            if isUTpresent == True or (isUTpresent == False and isvelovelo(utlistsz,velolistsz) == 'VeloVelo'):
                if isUTpresent == False:
                    eventconfig = 'VeloVelo'

                configvars = upstreamConfig(eventconfig)
                zmag=(upstreamConfig(eventconfig))[0]
                sigmax_nosm=upstreamConfig(eventconfig)[1]
                sigmay_nosm=upstreamConfig(eventconfig)[2]
                sigmatx_nosm=upstreamConfig(eventconfig)[3]
                sigmaty_nosm=upstreamConfig(eventconfig)[4]
                sigmax_wsm=upstreamConfig(eventconfig)[5]
                sigmay_wsm=upstreamConfig(eventconfig)[6]
                sigmatx_wsm=upstreamConfig(eventconfig)[7]
                sigmaty_wsm=upstreamConfig(eventconfig)[8]
                thr_wsm_list=upstreamConfig(eventconfig)[9]
                thr_nosm_list=upstreamConfig(eventconfig)[10]


                #change chi threshold, sigmas and windows depending on smearing on/off
                threshold_chi2_list = []

                if smearing == False:
                    for index in index_perc_list:
                        threshold_chi2_list.append(thr_nosm_list[index])
                    sigmax=sigmax_nosm
                    sigmay=sigmay_nosm
                    sigmatx=sigmatx_nosm
                    sigmaty=sigmaty_nosm

                else:
                    for index in index_perc_list:
                        threshold_chi2_list.append(thr_wsm_list[index])
                    sigmax=sigmax_wsm
                    sigmay=sigmay_wsm
                    sigmatx=sigmatx_wsm
                    sigmaty=sigmaty_wsm

                isLongTrack=False

                tlistsxyz = removezeros(event.HitXpos,event.HitYpos,event.HitZpos)
                tlistsx = tlistsxyz[0];tlistsy = tlistsxyz[1];tlistsz = tlistsxyz[2]




                #selecting some long tracks for event in ch1 and cut for !=0 else division/0
                if len(tlistsz)>1:

                    if (max(tlistsz)>min(tlistsz)>7000 and max(tlistsz)-min(tlistsz) != 0
                       and tlistsz[len(tlistsz)-1] == max(tlistsz) and tlistsz[0] == min(tlistsz)):
                        nLongTracks = nLongTracks+1
                        isLongTrack=True

                        #use fist hit in Velo and last in UT (todo: try first and last in Velo)
                        if eventconfig == 'VeloUT':
                            xv0=velolistsx[0];  yv0 = velolistsy[0]; zv0 = velolistsz[0];
                            xv1=utlistsx[len(utlistsz)-1];  yv1 = utlistsy[len(utlistsz)-1]; zv1 = utlistsz[len(utlistsz)-1];
                            if smearing == True:
                                xv1 = gRandom.Gaus(xv1,resx_mit)
                                yv1 = gRandom.Gaus(yv1,resy_mit)
                            #choose first and last hit in velo
                            '''if (velolistsz[len(velolistsz)-1]-zv0 != 0):
                                xv1=velolistsx[len(velolistsz)-1];  yv1 = velolistsy[len(velolistsz)-1]; zv1 = velolistsz[len(velolistsz)-1];
                            else:
                                xv1=utlistsx[len(utlistsz)-1];  yv1 = utlistsy[len(utlistsz)-1]; zv1 = utlistsz[len(utlistsz)-1];
                                eventconfig = 'VeloVelo'
                                if smearing == True:
                                    xv1 = gRandom.Gaus(xv1,resx_mit)
                                    yv1 = gRandom.Gaus(yv1,resy_mit)'''
                        #use fist and last hits in UT
                        elif eventconfig == 'UTUT':
                            xv0=utlistsx[0];  yv0 = utlistsy[0]; zv0 = utlistsz[0];
                            xv1=utlistsx[len(utlistsz)-1];  yv1 = utlistsy[len(utlistsz)-1]; zv1 = utlistsz[len(utlistsz)-1];
                            if smearing == True:
                                xv0 = gRandom.Gaus(xv0,resx_mit) ;xv1 = gRandom.Gaus(xv1,resx_mit)
                                yv0 = gRandom.Gaus(yv0,resy_mit) ;yv1 = gRandom.Gaus(yv1,resy_mit) 
                        #use fist and last hits in Velo
                        elif eventconfig == 'VeloVelo':
                            xv0=velolistsx[0];  yv0 = velolistsy[0]; zv0 = velolistsz[0];
                            xv1=velolistsx[len(velolistsz)-1];  yv1 = velolistsy[len(velolistsz)-1]; zv1 = velolistsz[len(velolistsz)-1];
                        else:
                            xv0=0;xv1=0;yv0=0;yv1=0;zv0=0;zv1=0;
                            break





                        xt0=tlistsx[0];  yt0 = tlistsy[0]; zt0 = min(tlistsz);
                        xt1=tlistsx[len(tlistsz)-1];  yt1 = tlistsy[len(tlistsz)-1]; zt1 = max(tlistsz);

                        vp = event.p

                        chi2real_l = ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, sigmax, sigmay, sigmatx, sigmaty, check_if_inwindow=False,smearing=smearing)
                        chi2real = chi2real_l[0]
                        entryreal = event.GetReadEntry()
                        eventreal = event.eventNumber
                        p = event.p
                        xzt0 = xt0 
                        yzt0 = yt0
                        dx = chi2real_l[1]
                        dy = chi2real_l[2]
                        dtx = chi2real_l[3]
                        dty = chi2real_l[4]



                #loop over all events, cut for momentum and dz !=0 else division/0 (later cut in window)
                for event2 in ch2:

                    tlistsxyz = removezeros(event2.HitXpos,event2.HitYpos,event2.HitZpos)
                    tlistsx = tlistsxyz[0];tlistsy = tlistsxyz[1];tlistsz = tlistsxyz[2]
                    
                    


                    if (len(tlistsz) > 1) and (isUTpresent==False or (isUTpresent==True and eventconfig != 'VeloVelo')):

                        #select T tracks in same event
                        if (event2.eventNumber == eventreal and max(tlistsz)>7000 and min(tlistsz)>7000 and max(tlistsz)-min(tlistsz) != 0):
                            
                            indexmax = tlistsz.index(max(tlistsz))
                            indexmin = tlistsz.index(min(tlistsz))
                                                    

                            #overwrite the t track coordinates (velo coordinates and momentum (?) remain the same)
                            xt0=tlistsx[indexmin];  yt0 = tlistsy[indexmin]; zt0 = tlistsz[indexmin];
                            xt1=tlistsx[indexmax];  yt1 = tlistsy[indexmax]; zt1 = tlistsz[indexmax];
                            
                            chi2try_l = ChiSquareVeloT(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1 , zmag, sigmax, sigmay, sigmatx, sigmaty, check_if_inwindow=False, smearing=smearing)
                            
                            if chi2try_l[0] != -1: #cut events in search window (see def of chi2 function)
                                ChisquareTry.append(chi2try_l[0])
                                Chi2Try_entryNumbers.append(event2.GetReadEntry())
                                pTry.append(event2.p)
                                xzt0Try.append(xt0)
                                yzt0Try.append(yt0)
                                dxTry.append(chi2try_l[1])
                                dyTry.append(chi2try_l[2])
                                dtxTry.append(chi2try_l[3])
                                dtyTry.append(chi2try_l[4])

                if len(ChisquareTry) != 0: #(else it means that there was no t track in the windows)

                    #STRATEGY MINSQUARE: find min chi2
                    ChisquareSM=min(ChisquareTry)
                    indexMin= ChisquareTry.index(ChisquareSM)

                    for threshold_chi2 in threshold_chi2_list:
                        index_chi2thr = threshold_chi2_list.index(threshold_chi2)

                        if ChisquareSM < threshold_chi2:
                            #check if it real or fake match (compare number of entry)
                            if (Chi2Try_entryNumbers[indexMin]==entryreal and isLongTrack==True):
                                real[index_chi2thr]+=1.
                                if(len(threshold_chi2_list)==1):
                                    #fill all the real plots
                                    chi2R_true_list.append(chi2real)
                                    pR_true_list.append(p)
                                    xzt0R_true_list.append(xzt0)
                                    yzt0R_true_list.append(yzt0)
                                    dxR_true_list.append(dx)
                                    dyR_true_list.append(dy)
                                    dtxR_true_list.append(dtx)
                                    dtyR_true_list.append(dty)

                            else:
                                fake[index_chi2thr]+=1.
                                if(len(threshold_chi2_list)==1):
                                    #fill all the real plots
                                    chi2R_list.append(chi2real)
                                    pR_list.append(p)
                                    xzt0R_list.append(xzt0)
                                    yzt0R_list.append(yzt0)
                                    dxR_list.append(dx)
                                    dyR_list.append(dy)
                                    dtxR_list.append(dtx)
                                    dtyR_list.append(dty)
                                    #fill all the fake plots
                                    chi2F_list.append(ChisquareSM)
                                    pF_list.append(pTry[indexMin])
                                    xzt0F_list.append(xzt0Try[indexMin])
                                    yzt0F_list.append(yzt0Try[indexMin])
                                    dxF_list.append(dxTry[indexMin])
                                    dyF_list.append(dyTry[indexMin])
                                    dtxF_list.append(dtxTry[indexMin])
                                    dtyF_list.append(dtyTry[indexMin])


                        #STRATEGY CUT BELOW THRESHOLD

                        chi_below_thr_perev = 0.
                        fakebelow_perev = 0.
                        realbelow_perev = 0.
                        count_perev = 0.

                        for chitry in ChisquareTry:
                            count_perev +=1
                            indexchitry = ChisquareTry.index(chitry)
                            if (chitry <= threshold_chi2):
                                #find how many tracks you find below threshold
                                chi_below_thr_perev+=1.
                                chi_below_thr[index_chi2thr]+=1.
                                #find how many times the real one is amongst those below threshold
                                if (Chi2Try_entryNumbers[indexchitry]==entryreal and isLongTrack==True):
                                    realbelow_perev+=1
                                    realbelow[index_chi2thr]+=1
                                #find how many wrong ones are below the threshold:
                                else:
                                    fakebelow_perev+=1
                                    fakebelow[index_chi2thr]+=1
                        if(len(threshold_chi2_list)==1):
                            #plot some kind of (un)efficiency per event 
                            if chi_below_thr != 0:
                                percF_th_list.append(fakebelow_perev)
                            #plot % of t tracks below threshold 
                            if len(ChisquareTry) != 0:
                                percbelow_th_list.append(chi_below_thr_perev/len(ChisquareTry))


                    else: #if chi2 is above threshold
                            if (Chi2Try_entryNumbers[indexMin]==entryreal and isLongTrack==True):
                                missed_real[index_chi2thr]+=1.
                            else:
                                missed_fake[index_chi2thr]+=1.
                if nLongTracks%100 == 0:
                    print nLongTracks
                    print (time.time() - timestart)/60, ' minutes'

                #cut at n good events (computational time issue)
                if (nLongTracks == 1 or entryreal == nentries):
                    break



#### PRINT RESULTS ####

print 'number of VeloUT tracks: ', nVeloUTTracks
print 'number of UTUT tracks: (downstream?) ', nUTUTTracks
print 'number of VeloVelo tracks: ', nVeloTracks
print 'number of upstream tracks', nUPTracks
print 'number of long tracks: ', nLongTracks

efficiency_list = []
efficiency_err_list = []
ghostrate_list = []
ghostrate_err_list = []
belowstats_list = []

for threshold_chi2 in threshold_chi2_list:
    index_chi2thr = threshold_chi2_list.index(threshold_chi2)
    efficiency = real[index_chi2thr]/nLongTracks
    ghostrate = fake[index_chi2thr]/(fake[index_chi2thr]+real[index_chi2thr])
    belowstats = realbelow[index_chi2thr]/(realbelow[index_chi2thr]+fakebelow[index_chi2thr])
    efficiency_list.append(efficiency)
    ghostrate_list.append(ghostrate)
    belowstats_list.append(belowstats)
    efficiency_err_list.append(np.sqrt(efficiency*(1-efficiency)/nLongTracks))
    ghostrate_err_list.append(np.sqrt(ghostrate*(1-ghostrate)/((fake[index_chi2thr]+real[index_chi2thr]))))


deltat = time.time() - timestart
print '\ntime: ', deltat/60

print 'percentages: ', percentage_list 
print 'thresholds chi2: ', threshold_chi2_list
print 'efficiencies: ', efficiency_list
print 'efficiencies errors: ', efficiency_err_list
print 'ghost rates: ', ghostrate_list
print 'ghost rates errors: ', ghostrate_err_list
print 'belowstats: ', belowstats_list



smearing:  True
is UT present:  True
number of VeloUT tracks:  1
number of UTUT tracks: (downstream?)  0
number of VeloVelo tracks:  0
number of upstream tracks 1
number of long tracks:  1

time:  0.689882202943
percentages:  [99, 97, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 73, 71, 69, 67, 65, 63, 61]
thresholds chi2:  [3348.71, 945.25, 493.35, 309.55, 192.49, 124.37, 89.42, 62.38, 46.07, 35.87, 25.4, 18.44, 14.19, 10.61, 7.82, 5.68, 4.39, 3.38, 2.68, 2.04]
efficiencies:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
efficiencies errors:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ghost rates:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ghost rates errors:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
belowstats:  [0.0013679890560875513, 0.002352941176470588,

In [ ]:
Nbins=60

#gROOT.ProcessLine(".x ~/lhcbStyle.C")
#gStyle.SetPaintTextFormat("1.3f")
gStyle.SetOptStat(1110) #nentries, mean, rms
#gStyle.SetOptStat("") 

rangex = sigmax*2
rangey = sigmay*2
rangetx = sigmatx*2
rangety = sigmaty*2
        
#plots for strategy min chi2
#chi2R_true=TH1F("chi2R_true","",Nbins,0,threshold_chi2)
#chi2R=TH1F("chi2R","",Nbins,0,threshold_chi2)
#chi2F=TH1F("chi2F","",Nbins,0,threshold_chi2)

chi2R_true=TH1F("chi2R_true","",Nbins,0,1)
chi2R=TH1F("chi2R","",Nbins,0,1)
chi2F=TH1F("chi2F","",Nbins,0,1)


pR_true=TH1F("pF_true","",Nbins,5000,50000)
pR=TH1F("pR","",Nbins,5000,50000)
pF=TH1F("pF","",Nbins,5000,50000)

xyzt0R_true=TH2F("xyzt0R_true","",Nbins,-3000,+3000,Nbins,-3000,+3000)
xyzt0R=TH2F("xyzt0R","",Nbins,-3000,+3000,Nbins,-3000,+3000)
xyzt0F=TH2F("xyzt0F","",Nbins,-3000,+3000,Nbins,-3000,+3000)

dxR_true=TH1F("dxR_true","",Nbins,-rangex,rangex)
dxR=TH1F("dxR","",Nbins,-rangex,rangex)
dxF=TH1F("dxF","",Nbins,-rangex,rangex)

dyR_true=TH1F("dyR_true","",Nbins,-rangey,rangey)
dyR=TH1F("dyR","",Nbins,-rangey,rangey)
dyF=TH1F("dyR","",Nbins,-rangey,rangey)

dtxR_true=TH1F("dtxR_true","",Nbins,-rangetx,rangetx)
dtxR=TH1F("dtxR","",Nbins,-rangetx,rangetx)
dtxF=TH1F("dtxF","",Nbins,-rangetx,rangetx)

dtyR_true=TH1F("dtyR_true","",Nbins,-rangety,rangety)
dtyR=TH1F("dtyR","",Nbins,-rangety,rangety)
dtyF=TH1F("dtyF","",Nbins,-rangety,rangety)


#plots for strategy cut below threshold
percbelow_th=TH1F("chi2R_th","",Nbins,0,1)
percF_th=TH1F("chi2F_th","",Nbins,0,30)

In [143]:
#list of lists
listlist = [chi2R_true_list,chi2R_list,chi2F_list,pR_true_list,pR_list,pF_list,dxR_true_list,dxR_list,dxF_list,dyR_true_list,dyR_list,dyF_list,dtxR_true_list,dtxR_list,dtxF_list,dtyR_true_list,dtyR_list,dtyF_list,percbelow_th_list,percF_th_list]
#list of th1f histos
th1flist = [chi2R_true,chi2R,chi2F,pR_true,pR,pF,dxR_true,dxR,dxF,dyR_true,dyR,dyF,dtxR_true,dtxR,dtxF,dtyR_true,dtyR,dtyF,percbelow_th,percF_th]

#fill every th1f with the lists
for singlehisto in th1flist:
    index = th1flist.index(singlehisto)
    singlelist = listlist[index]
    for item in singlelist:
        singlehisto.Fill(item)



#list of lists
listlist2 = [xzt0R_true_list,yzt0R_true_list,xzt0R_list,yzt0R_list,xzt0F_list,yzt0F_list]
#list of th2f histos
th2flist = [xyzt0R_true,xyzt0R,xyzt0F]

#fill every histogram with the lists
for singlehisto in th2flist:
    index_x = (th2flist.index(singlehisto))*2
    index_y = (th2flist.index(singlehisto))*2+1
    singlelist_x = listlist2[index_x]
    singlelist_y = listlist2[index_y]
    for index in range(0,len(singlelist_x)):
        singlehisto.Fill(singlelist_x[index],singlelist_y[index])


In [135]:
#draw plots for minimum chi2 strategy

c4=TCanvas("c4","",2000,2000)

c4.Divide(3,3)

c4.cd(1)
chi2R.Draw()
chi2R.GetXaxis().SetTitle("#chi^{2}")
chi2R.GetYaxis().SetTitle("Tracks")
chi2R.GetYaxis().SetTitleOffset(0.8)
chi2R.SetLineColor(kBlue)
chi2R.SetMinimum(0.)
chi2F.Draw("SAME")
chi2F.SetLineColor(kRed)

legendi=TLegend(0.2,0.75,0.5,0.9)
legendi.AddEntry("chi2R","min = fake; plot real","f")
legendi.AddEntry("chi2F","min = fake; plot fake","f")
legendi.SetTextSize(.04)
legendi.Draw()

c4.cd(2)
pR.Draw()
pR.GetXaxis().SetTitle("p")
pR.GetYaxis().SetTitle("Tracks")
pR.SetLineColor(kBlue)
pF.Draw("SAME")
pF.SetLineColor(kRed)


c4.cd(3)
xyzt0R.Draw()
xyzt0R.GetXaxis().SetTitle("x")
xyzt0R.GetYaxis().SetTitle("y")
xyzt0R.SetMarkerColor(kBlue)
xyzt0F.Draw("SAME")
xyzt0F.SetMarkerColor(kRed)



c4.cd(4)
dxR.Draw()
dxR.GetXaxis().SetTitle("dx")
dxR.GetYaxis().SetTitle("Tracks")
dxR.SetLineColor(kBlue)
dxF.Draw("SAME")
dxF.SetLineColor(kRed)



c4.cd(5)

dyR.Draw()
dyR.GetXaxis().SetTitle("dy")
dyR.GetYaxis().SetTitle("Tracks")
dyR.SetLineColor(kBlue)
dyF.Draw("SAME")
dyF.SetLineColor(kRed)


c4.cd(6)
dtxR.Draw()
dtxR.GetXaxis().SetTitle("dtx")
dtxR.GetYaxis().SetTitle("Tracks")
dtxR.SetLineColor(kBlue)
dtxF.Draw("SAME")
dtxF.SetLineColor(kRed)


c4.cd(7)
dtyR.Draw()
dtyR.GetXaxis().SetTitle("dty")
dtyR.GetYaxis().SetTitle("Tracks")
dtyR.SetLineColor(kBlue)
dtyF.Draw("SAME")
dtyF.SetLineColor(kRed)


c4.SaveAs("distinguish_range_fr_fakemin_firstlastcuts_97sm_nocut800.png")



Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Info in <TCanvas::Print>: png file distinguish_range_fr_fakemin_firstlastcuts_97sm_nocut800.png has been created


In [136]:

#draw plots for minimum chi2 strategy

c3=TCanvas("c3","",2000,2000)

c3.Divide(3,3)

c3.cd(1)
chi2R_true.Draw()
chi2R_true.GetXaxis().SetTitle("#chi^{2}")
chi2R_true.GetYaxis().SetTitle("Tracks")
chi2R_true.GetYaxis().SetTitleOffset(0.8)
chi2R_true.SetLineColor(kBlue)
chi2R_true.SetMinimum(0.)
legendi=TLegend(0.2,0.75,0.5,0.9)
legendi.AddEntry("chi2R_true","min = true; plot real","f")
legendi.SetTextSize(.04)
legendi.Draw()

c3.cd(2)
pR_true.Draw()
pR_true.GetXaxis().SetTitle("p")
pR_true.GetYaxis().SetTitle("Tracks")
pR_true.SetLineColor(kBlue)

c3.cd(3)
xyzt0R_true.Draw()
xyzt0R_true.GetXaxis().SetTitle("x")
xyzt0R_true.GetYaxis().SetTitle("y")
xyzt0R.SetMarkerColor(kBlue)


c3.cd(4)
dxR_true.Draw()
dxR_true.GetXaxis().SetTitle("dx")
dxR_true.GetYaxis().SetTitle("Tracks")
dxR_true.SetLineColor(kBlue)

c3.cd(5)
dyR_true.Draw()
dyR_true.GetXaxis().SetTitle("dy")
dyR_true.GetYaxis().SetTitle("Tracks")
dyR_true.SetLineColor(kBlue)

c3.cd(6)
dtxR_true.Draw()
dtxR_true.GetXaxis().SetTitle("dtx")
dtxR_true.GetYaxis().SetTitle("Tracks")
dtxR_true.SetLineColor(kBlue)

c3.cd(7)
dtyR_true.Draw()
dtyR_true.GetXaxis().SetTitle("dty")
dtyR_true.GetYaxis().SetTitle("Tracks")
dtyR_true.SetLineColor(kBlue)

c3.SaveAs("distinguish_range_fr_truemin_firstlastcuts_97sm_nocut800.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file distinguish_range_fr_truemin_firstlastcuts_97sm_nocut800.png has been created


In [137]:

#draw plots for cut below threshold strategy

c5=TCanvas("c5","",1300,900)
c5.Divide(2,1)
c5.cd(1)
percF_th.Draw()
percF_th.GetXaxis().SetTitle("fakebelow")
percF_th.GetYaxis().SetTitle("Tracks")
percF_th.GetYaxis().SetTitleOffset(0.8)
percF_th.SetLineColor(kBlue)
percF_th.SetMinimum(0.)

c5.cd(2)
percbelow_th.Draw()
percbelow_th.GetXaxis().SetTitle("totbelow/chi2try")
percbelow_th.GetYaxis().SetTitle("Tracks")
percbelow_th.GetYaxis().SetTitleOffset(0.8)
percbelow_th.SetLineColor(kRed)
percbelow_th.SetMinimum(0.)
c5.SaveAs("distinguish_range_belowthr_firstlastcuts_97sm_nocut800.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Info in <TCanvas::Print>: png file distinguish_range_belowthr_firstlastcuts_97sm_nocut800.png has been created


In [138]:
def cloneandscale(th1f,scale=1):
    scaledhisto = (th1f.Clone(""))
    for iteration in range(0,scale-1):
        scaledhisto.Add(th1f)
    return scaledhisto


In [144]:
#draw plots for minimum chi2 strategy

gStyle.SetOptStat("") #nothing


scalingfactor = 1

c7=TCanvas("c7","",2000,2000)

c7.Divide(3,3)

c7.cd(1)
chi2R_true.Draw()
chi2R_true.GetXaxis().SetTitle("#chi^{2}")
chi2R_true.GetYaxis().SetTitle("Tracks")
chi2R_true.GetYaxis().SetTitleOffset(0.8)
chi2R_true.SetLineColor(kGreen)
chi2R_true.SetMinimum(0.)
chi2Rscaled = cloneandscale(chi2R,scalingfactor)
chi2Rscaled.Draw("SAME")
chi2Rscaled.SetLineColor(kBlue)
chi2Fscaled = cloneandscale(chi2R,scalingfactor)
chi2Fscaled.Draw("SAME")
chi2Fscaled.SetLineColor(kRed)


legendi=TLegend(0.3,0.75,0.5,0.9)
legendi.AddEntry(chi2Rscaled,"min = fake; plot real","f")
legendi.AddEntry(chi2Fscaled,"min = fake; plot fake","f")
legendi.AddEntry("chi2R_true","min = real; plot real","f")
legendi.SetTextSize(.04)
legendi.Draw()

c7.cd(2)
pR_true.Draw()
pR_true.GetXaxis().SetTitle("p")
pR_true.GetYaxis().SetTitle("Tracks")
pR_true.SetLineColor(kGreen)
pRscaled = cloneandscale(pR,scalingfactor)
pRscaled.Draw("SAME")
pRscaled.SetLineColor(kBlue)
pFscaled = cloneandscale(pF,scalingfactor)
pFscaled.Draw("SAME")
pFscaled.SetLineColor(kRed)



c7.cd(3)
xyzt0R_true.Draw()
xyzt0R_true.GetXaxis().SetTitle("x")
xyzt0R_true.GetYaxis().SetTitle("y")
xyzt0R_true.SetMarkerColor(kGreen)
xyzt0Rscaled = cloneandscale(xyzt0R,scalingfactor)
xyzt0Rscaled.Draw("SAME")
xyzt0Rscaled.SetMarkerColor(kBlue)
xyzt0Fscaled = cloneandscale(xyzt0F,scalingfactor)
xyzt0Fscaled.Draw("SAME")
xyzt0Fscaled.SetMarkerColor(kRed)


c7.cd(4)
dxR_true.Draw()
dxR_true.GetXaxis().SetTitle("dx")
dxR_true.GetYaxis().SetTitle("Tracks")
dxR_true.SetLineColor(kGreen)
dxRscaled = cloneandscale(dxR,scalingfactor)
dxRscaled.Draw("SAME")
dxRscaled.SetLineColor(kBlue)
dxFscaled = cloneandscale(dxF,scalingfactor)
dxFscaled.Draw("SAME")
dxFscaled.SetLineColor(kRed)


c7.cd(5)
dyR_true.Draw()
dyR_true.GetXaxis().SetTitle("dy")
dyR_true.GetYaxis().SetTitle("Tracks")
dyR_true.SetLineColor(kGreen)
dyRscaled = cloneandscale(dyR,scalingfactor)
dyRscaled.Draw("SAME")
dyRscaled.SetLineColor(kBlue)
dyFscaled = cloneandscale(dyF,scalingfactor)
dyFscaled.Draw("SAME")
dyFscaled.SetLineColor(kRed)


c7.cd(6)
dtxR_true.Draw()
dtxR_true.GetXaxis().SetTitle("dtx")
dtxR_true.GetYaxis().SetTitle("Tracks")
dtxR_true.SetLineColor(kGreen)
dtxRscaled = cloneandscale(dtxR,scalingfactor)
dtxRscaled.Draw("SAME")
dtxRscaled.SetLineColor(kBlue)
dtxFscaled = cloneandscale(dtxF,scalingfactor)
dtxFscaled.Draw("SAME")
dtxFscaled.SetLineColor(kRed)

c7.cd(7)
dtyR_true.Draw()
dtyR_true.GetXaxis().SetTitle("dty")
dtyR_true.GetYaxis().SetTitle("Tracks")
dtyR_true.SetLineColor(kGreen)
dtyRscaled = cloneandscale(dtyR,scalingfactor)
dtyRscaled.Draw("SAME")
dtyRscaled.SetLineColor(kBlue)
dtyFscaled = cloneandscale(dtyF,scalingfactor)
dtyFscaled.Draw("SAME")
dtyFscaled.SetLineColor(kRed)


c7.SaveAs("distinguish_range_fr_all_firstlastcuts_97sm_nocut800_sc1.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c7
Info in <TCanvas::Print>: png file distinguish_range_fr_all_firstlastcuts_97sm_nocut800_sc1.png has been created
